In [1]:
#input_stock_code = '公司：4Paradigm（06682.HK）' # 输入的信息
company_name = '商汤科技'  # 公司名称
company_code = '0020.HK'  # 公司代码
input_stock_code = f"{company_name}（{company_code}）" 

task1 = f"""请帮我生成公司/个股研报，目标公司和股票信息是 {input_stock_code}。
要求能够自动抽取三大会计报表与股权结构，输出主营业务、核心竞争力与行业地位；支持财务比率计算与行业对比分析（如ROE分解、毛利率、现金流匹配度），结合同行企业进行横向竞争分析；构建估值与预测模型，模拟关键变量变化对财务结果的影响（如原材料成本、汇率变动）；结合公开数据与管理层信息，评估公司治理结构与发展战略，提出投资建议与风险提醒。"""


In [2]:
from constants import *
from financial_data_search import *
from python_data_analyse import *
import akshare as ak
import pandas as pd
from get_capital_structure import get_capital_structure_cn_ths, get_capital_structure_hk_ths
from get_cn_control import get_cn_control
from get_company_field_compare import get_cn_company_field_compare, get_hk_company_field_compare
from get_company_finance_summary import get_company_finance_summary_cn,get_company_finance_summary_hk
from get_company_intro import get_company_profile_ths_cn, get_company_profile_ths_hk, get_cn_company_profile_ak, get_hk_company_profile_ak
from get_financial_data_annual import download_cn_financial_data, download_hk_financial_data
from get_rating_info import get_hk_rating_info, get_cn_rating_info
from get_stock_info import get_cn_stock_info, get_hk_stock_info
from get_worth_predict import get_cn_worth_predict, get_hk_worth_predict
from datetime import datetime, timedelta
import time
import requests
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from typing import Dict, Any
from typing_extensions import Annotated, TypedDict
import json
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage,SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from typing import Dict, Optional, Literal
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
import ast
import pickle

from typing import List

from langchain_core.tools import InjectedToolArg, tool
from typing_extensions import Annotated
import time
from time import sleep
from constants import *
# 支持matplotlib输出中文
import matplotlib.pyplot as plt
import chineseize_matplotlib
plt.rcParams['axes.unicode_minus'] = False # 解决负号'-'显示为方块的问题

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# 1. 数据采集函数

In [ ]:
######################## 0. 公司和股票信息
def extract_company_basic_info(stock_type,stock_code):
    """
    输入股票类型和股票代码，提取公司的基本信息
    Args:
        stock_type (str): 股票类型，'A股' 或 '港股'
        stock_code (str): 股票代码，A股为6位数字，港股为4位数字
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, str]): 公司基本信息数据值，包括公司名称、英文名称、主营业务、所属行业和公司简介
            data_type (str): 数据类型
            data_desc (str): 数据内容描述，说明返回的数据包含哪些信息

    """
    if stock_type == '港股':
        hk_code = 'HK' + stock_code
        company_info = get_company_profile_ths_hk(hk_code)
    else:
        a_code = stock_code
        company_info = get_company_profile_ths_cn(a_code)

    return {
        'data_value': {
            "公司名称": company_info['公司名称'],
            "英文名称": company_info['英文名称'],
            "主营业务": company_info['主营业务'],
            "所属行业": company_info['所属行业'],
            "公司简介": company_info['公司简介']
        },
        'data_type': 'dict',
        'data_desc': '公司基本信息，包括公司名称、英文名称、主营业务、所属行业和公司简介'
    }

# 提取公司基本信息
# company_basic_info = extract_company_basic_info(stock_type, stock_code)

## 获取最近的几个月，月度的股票数据
def get_date_range_months(months_back=6):
    end_date = datetime.today().replace(day=1)  # 当月1号
    start_date = end_date - timedelta(days=30*months_back)
    start_date = start_date.replace(day=1)  # 第一天
    return {
        "start_date": start_date.strftime("%Y%m%d"),
        "end_date": end_date.strftime("%Y%m%d")
    }

# start_date = get_date_range_months()["start_date"]
# end_date = get_date_range_months()["end_date"]

# if stock_type=='港股':
#     hk_code = 'HK'+stock_code
#     # 获取近6个月的股票信息，获取近6个月的1号的信息
#     stock_info = get_hk_stock_info(hk_code,period="monthly", start_date=start_date, end_date=end_date) # 参数可以控制日期
#     # 单位是港元
#     # ['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率']
# else:
#     a_code = stock_code
#     stock_info = get_cn_stock_info(a_code,period="monthly", start_date=start_date, end_date=end_date)
#     # 单位是人民币
#     # 日期	开盘	收盘	最高	最低	成交量	成交额	振幅	涨跌幅	涨跌额	换手率


def extract_stock_info(stock_type, stock_code,months_back=6):
    """
    提取公司股票的近期基本价格和成交信息，默认提取最近6个月的月度数据；
    返回值是一个字典，包含日期、开盘、收盘、最高、最低、成交量、成交额、振幅、涨跌幅、涨跌额和换手率等信息。
    返回结果的价格，对于港股单位是港元，对于A股单位是人民币。
    Args:
        stock_type (str): 股票类型，'A股' 或 '港股'
        stock_code (str): 股票代码，A股为6位数字，港股为4位数字
        months_back (int): 回溯的月数，默认为6个月
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_name (Dict[str, list]): 存放数据值，包含日期、开盘、收盘、最高、最低、成交量、成交额、振幅、涨跌幅、涨跌额和换手率
            data_type (Dict[str, str]): 数据类型
            data_desc (Dict[str, str]): 包含上述信息的内容描述
    """
    start_date = get_date_range_months(months_back)["start_date"]
    end_date = get_date_range_months(months_back)["end_date"]
    if stock_type == '港股':
        hk_code = 'HK' + stock_code
        stock_info = get_hk_stock_info(hk_code, period="monthly", start_date=start_date, end_date=end_date)
    else:
        a_code = stock_code
        stock_info = get_cn_stock_info(a_code, period="monthly", start_date=start_date, end_date=end_date)

    return {
        'data_value': {
            "股票近期价格和交易信息": stock_info
    },
        'data_type': 'dict_dataframe',
        'data_desc': '包含股票的日期、开盘价、收盘价、最高价、最低价、成交量、成交额、振幅、涨跌幅、涨跌额和换手率等信息，通过key值访问list数据'
    }

# 提取股票信息
# stock_info = extract_stock_info(stock_type, stock_code)


################## 1. 获取股本结构

def extract_capital_structure(stock_type, stock_code):
    """
    提取股本结构信息
    Args:
        stock_type (str): 股票类型，'A股' 或 '港股'
        stock_code (str): 股票代码，A股为6位数字，港股为4位数字
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, list]): 股本结构数据值，港股的包括总股本、港股总股本、优先股和变动日期；A股包括总股本、A股总股本、流通A股、限售A股和变动原因
            data_type (str): 数据类型
            data_desc (str): 数据内容描述，说明返回的数据包含哪些信息
    """
    if stock_type == '港股':
        hk_code = 'HK' + stock_code
        captial_structure = get_capital_structure_hk_ths(hk_code)
        return {
            'data_value': {
                "港股股本": captial_structure,
            },
            'data_type': 'dict_dataframe',
            'data_desc': '对应港股的股本结构信息，包括总股本、港股总股本、优先股和变动日期'
        }
    else:
        a_code = stock_code
        cn_captial_structure = get_capital_structure_cn_ths(a_code)
        # return {
        #     "总股本(股)": cn_captial_structure['总股本(股)'],
        #     "A股总股本(股)": cn_captial_structure['A股总股本(股)'],
        #     "流通A股(股)": cn_captial_structure['流通A股(股)'],
        #     "限售A股(股)": cn_captial_structure['限售A股(股)'],
        #     "变动原因": cn_captial_structure['变动原因']
        # }
        return {
            'data_value': {
                "A股股本": cn_captial_structure
            },
            'data_type': 'dict_dataframe',
            'data_desc': '对应A股的股本结构信息，包括总股本、A股总股本、流通A股、限售A股和变动原因'
        }

# 提取股本结构信息
# capital_structure = extract_capital_structure(stock_type, stock_code)


# ##资产负债表：反映企业在特定日期的财务状况，包括资产、负债和所有者权益。
# ##利润表：展示企业在一定会计期间的经营成果，反映企业的收入、费用和利润。
# ##现金流量表：记录企业在特定期间内的现金流入和流出，反映企业的现金流动情况。


# TODO--这个是个大表，看一下到底需要哪些列的信息
def extract_financial_statements(stock_type, stock_code):
    """
    提取年度三大会计报表：资产负债表、利润表和现金流量表
    返回值是一个字典，包含三个DataFrame，分别对应资产负债表、利润表和现金流量表。
    Args:
        stock_type (str): 股票类型，'A股' 或 '港股'
        stock_code (str): 股票代码，A股为6位数字，港股为4位数字
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, pd.DataFrame]): 包含三个DataFrame的字典，分别是资产负债表、利润表和现金流量表
            data_type (str): 数据类型
            data_desc (str): 数据内容描述，说明返回的数据包含哪些信息
    """
    if stock_type == '港股':
        hk_code = 'HK' + stock_code
        hk_code = hk_code.upper()
        hk_code = ''.join(filter(str.isdigit, hk_code))  # 只保留数字
        if len(hk_code) > 4:
            hk_code = hk_code[-4:]
        hk_code = '0' + hk_code  # 确保是5位数字
        stock_financial_bi_df = ak.stock_financial_hk_report_em(stock=hk_code, symbol="资产负债表", indicator="年度")
        stock_benefit_bi_df = ak.stock_financial_hk_report_em(stock=hk_code, symbol="利润表", indicator="年度")
        stock_cash_bi_df = ak.stock_financial_hk_report_em(stock=hk_code, symbol="现金流量表", indicator="年度")
    else:
        a_code = stock_code
        stock_financial_bi_df = ak.stock_financial_debt_ths(symbol=a_code, indicator="按年度")
        stock_benefit_bi_df = ak.stock_financial_benefit_ths(symbol=a_code, indicator="按年度")
        stock_cash_bi_df = ak.stock_financial_cash_ths(symbol=a_code, indicator="按年度")

    return {
        'data_value': {
            "资产负债表": stock_financial_bi_df,
            "利润表": stock_benefit_bi_df,
            "现金流量表": stock_cash_bi_df
        },
        'data_type': 'dict_dataframe',
        'data_desc': '包含对应股票代码的会计报表'
    }

# 提取年度三大会计报表
# financial_statements = extract_financial_statements(stock_type, stock_code)



########### 4. 港股ROE数据分析


def extract_hk_financial_analysis(stock_type, stock_code):
    """
    提取港股的财务指标分析数据
    Args:
        stock_type (str): 股票类型，'港股'
        stock_code (str): 港股股票代码，4位数字
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, pd.DataFrame]): 港股财务分析指标数据，包含ROE等指标
            data_type (str): 数据类型
            data_desc (str): 数据内容描述，说明返回的数据包含哪些财务指标分析数据
    """
    if stock_type == '港股':
        hk_code = 'HK' + stock_code
        hk_code = hk_code.upper()
        hk_code = ''.join(filter(str.isdigit, hk_code))  # 只保留数字
        if len(hk_code) > 4:
            hk_code = hk_code[-4:]
        hk_code = '0' + hk_code  # 确保是5位数字
        stock_financial_analysis_indicator_em_df = ak.stock_financial_hk_analysis_indicator_em(symbol=hk_code, indicator="年度")
        return {
            'data_value': {'港股财务分析指标数据':stock_financial_analysis_indicator_em_df},
            'data_type': 'dict_dataframe',
            'data_desc': '港股的财务指标分析数据，包括ROE等指标'
        }
    else:
        return None
    
# 提取港股的财务指标分析数据
# hk_financial_analysis = extract_hk_financial_analysis(stock_type, stock_code)


############# 5. 获取财务摘要信息总结

def extract_financial_summary(stock_type, stock_code):
    """
    提取公司财务摘要信息
    Args:
        stock_type (str): 股票类型，'A股' 或 '港股'
        stock_code (str): 股票代码，A股为6位数字，港股为4位数字
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, str]): 公司财务摘要信息
            data_type (str): 数据类型
            data_desc (str): 数据内容描述，说明返回的数据包含哪些信息
    """
    if stock_type == '港股':
        hk_code = 'HK' + stock_code
        official_finance_summary = get_company_finance_summary_hk(hk_code)
    else:
        a_code = stock_code
        official_finance_summary = get_company_finance_summary_cn(a_code)

    return {
        'data_value': {
            "财务摘要": official_finance_summary
        },
        'data_type': 'dict',
        'data_desc': '公司财务信息的摘要分析信息，包括业绩回顾、展望和竞争力分析等内容'
    }
# 提取财务摘要信息
# financial_summary = extract_financial_summary(stock_type, stock_code)


################### 6. 获取行业对比信息

def extract_field_compare(stock_type, stock_code):
    """
    提取行业对比分析数据
    Args:
        stock_type (str): 股票类型
        stock_code (str): 股票代码
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, dict]): 行业对比分析数据，包含目标公司和同行业其他公司的对比数据
            data_type (str): 数据类型
            data_desc (str): 数据内容描述，说明返回的数据包含哪些行业对比分析数据
    """
    if stock_type == '港股':
        hk_code = 'HK' + stock_code
        field_compare = get_hk_company_field_compare(hk_code)  # 港股包括
    else:
        a_code = stock_code
        field_compare = get_cn_company_field_compare(a_code)

    issue_list = []
    title_list = []
    ps_list = []
    dataframe_list = []
    for item in field_compare:
        issue_list.append(item['issue'])
        title_list.append(item['title'])
        ps_list.append(item['ps'])
        dataframe_list.append(item['table'])
    return {
        'data_value':{
            "issue": issue_list,
            "title": title_list,
            "ps": ps_list,
            "table": dataframe_list
        },
        'data_type': 'dict_special_list',
        'data_desc': '行业对比分析数据，包含目标公司和同行业其他公司的对比数据，是一个list，包括issue，title，ps和数据table，table是原始数据dataframe'
    }


# 提取行业对比分析数据
# field_compare_data = extract_field_compare(stock_type, stock_code)

################# 7. 综合分析，展望未来
def extract_rating_info(stock_type: Annotated[str, InjectedToolArg], stock_code: Annotated[str, InjectedToolArg]):
    """
    提取评级信息
    Args:
        stock_type (str): 股票类型，'A股' 或 '港股'
        stock_code (str): 股票代码，A股为6位数字，港股为4位数字
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, Any]): 评级信息，包括评级机构的评级和投资建议
            data_type (str): 数据类型
            data_desc (str): 数据内容描述，说明返回的数据包含哪些评级信息
    """
    if stock_type == '港股':
        hk_code = 'HK' + stock_code
        rating_info = get_hk_rating_info(hk_code)
    else:
        a_code = stock_code
        rating_info = ak.stock_profit_forecast_ths(symbol=a_code, indicator="业绩预测详表-机构")

    return {
        'data_value': {
            "评级信息": rating_info
        },
        'data_type': 'dict_dataframe',
        'data_desc': '评级信息，包括评级机构的评级和投资建议，主要关注机构和研究员给出的数据指标是否建议买入'
    }

# 提取评级信息
# rating_info = extract_rating_info(stock_type, stock_code)


################## 8.未来和价值预测信息



def extract_worth_predict(stock_type: Annotated[str, InjectedToolArg], stock_code: Annotated[str, InjectedToolArg]):
    """
    提取股票收益和价值预测信息
    Args:
        stock_type (str): 股票类型，'A股' 或 '港股'
        stock_code (str): 股票代码，A股为6位数字，港股为4位数字
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, pd.DataFrame]): 股票收益和价值预测信息，包括盈利预测概览和详细指标预测的数据值
            data_type (str): 数据类型
            data_desc (str): 数据内容描述，说明返回的数据包含哪些股票收益和价值预测信息
    """
    if stock_type == '港股':
        hk_code = 'HK' + stock_code
        hk_code = hk_code.upper()
        hk_code = ''.join(filter(str.isdigit, hk_code))  # 只保留数字
        if len(hk_code) > 4:
            hk_code = hk_code[-4:]
        hk_code = '0' + hk_code  # 确保是5位数字
        predict_info = ak.stock_hk_profit_forecast_et(symbol=hk_code, indicator="盈利预测概览")
    else:
        a_code = stock_code
        predict_info = ak.stock_profit_forecast_ths(symbol=a_code, indicator="业绩预测详表-详细指标预测")

    return {
        'data_value': {
            "预测信息": predict_info
        },
        'data_type': 'dict_dataframe',
        'data_desc': '股票收益和价值预测信息，包括盈利预测概览和详细指标预测的数据值'
    }

# 提取股票收益和价值预测信息
# worth_predict = extract_worth_predict(stock_type, stock_code)

# 2. 数据获取和下载

In [ ]:
# 信息获取后，数据存储到dict里面
collected_data_value = {}
collected_data_type = {}
collected_data_desc = {}

In [ ]:
def save_extracted_info(data_value,
                        data_type,
                        data_desc):
    global collected_data_value, collected_data_type, collected_data_desc
    if data_type=='dict': # 普通的dict直接放进去就可以了
        collected_data_value.update(data_value)
        # 注意提取出data_value里面的key作为后面的数据类型和描述的key
        for key in data_value.keys():
            collected_data_type[key] = 'str'
            collected_data_desc[key] = str(key)
    elif data_type=='dict_dataframe':
        collected_data_value.update(data_value)
        # 注意提取出data_value里面的key作为后面的数据类型和描述的key
        for key in data_value.keys():
            collected_data_type[key] = 'dataframe'
            collected_data_desc[key] = data_desc
    elif data_type=='dict_special_list': # 最复杂的一种，这个的data_value是一个list
        for i in range(len(data_value['issue'])):
            issue = data_value['issue'][i]
            title = data_value['title'][i]
            ps = data_value['ps'][i]
            dataframe = data_value['table'][i]
            description = f"{issue} - {title} - ({ps})"
            if '行业分类' in title:
                key = str(issue) + '行业财务数据'
            else:
                if '价值' in title or '表现' in title:
                    key = str(issue) + '行业价值表现数据'
                else:
                    key = str(issue) + '行业财务经营数据'
            collected_data_value[key] = dataframe
            collected_data_type[key] = 'dataframe'
            collected_data_desc[key] = description


import pickle

def save_collected_data_to_local():
    with open('collected_data_value.pkl', 'wb') as f:
        pickle.dump(collected_data_value, f)
    with open('collected_data_type.pkl', 'wb') as f:
        pickle.dump(collected_data_type, f)
    with open('collected_data_desc.pkl', 'wb') as f:
        pickle.dump(collected_data_desc, f)

    print("数据已成功保存到本地。")

def load_collected_data_from_local():
    global collected_data_value, collected_data_type, collected_data_desc
    try:
        with open('collected_data_value.pkl', 'rb') as f:
            collected_data_value = pickle.load(f)
        with open('collected_data_type.pkl', 'rb') as f:
            collected_data_type = pickle.load(f)
        with open('collected_data_desc.pkl', 'rb') as f:
            collected_data_desc = pickle.load(f)
        print("数据已从本地加载。")
    except FileNotFoundError:
        print("未找到本地数据文件，请先运行保存操作。")


In [ ]:
def judge_stock_type(input_stock_code: str):
    """
    调用该函数，从混合着股票代码和公司信息的输入判断股票是A股的还是港股的，并提取出准确的股票代码。
    Args:
        input_stock_code (str): 输入的股票代码和公司信息
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, str]): 包含股票类型（A股或港股）和对应的股票代码
            data_type (str): 数据类型
            data_desc (str): 数据描述
    例如：
    """
    prompt = """你是一个股票研究专家，请判断这个股票代码是A股还是港股，并返回股票代码。对于A股，返回6位纯数字的股票代码；对于港股，返回4位纯数字的股票代码。
    股票代码和公司信息是：{stock_code}
    请只返回股票代码，不要包含其他任何信息。你输出的股票代码为："""
    response = qwen_plus.invoke(prompt.format(stock_code=input_stock_code),enable_thinking=False)
    stock_code = response.content.strip()
    if len(stock_code) == 6:
        stock_type = 'A股'
    else:
        stock_type = '港股'
    return stock_type, stock_code
stock_type, stock_code = judge_stock_type(input_stock_code)

In [ ]:
# 2. 提取这个公司的基本信息
company_basic_info = extract_company_basic_info(stock_type, stock_code)
save_extracted_info(
    data_value=company_basic_info['data_value'],
    data_type=company_basic_info['data_type'],
    data_desc=company_basic_info['data_desc']
)

# 3. 提取近期股票基本数据
stock_info = extract_stock_info(stock_type, stock_code)

save_extracted_info(
    data_value=stock_info['data_value'],
    data_type=stock_info['data_type'],
    data_desc=stock_info['data_desc']
)

# 4. 提取三大财务报表数据
financial_statements = extract_financial_statements(stock_type, stock_code)

save_extracted_info(
    data_value=financial_statements['data_value'],
    data_type=financial_statements['data_type'],
    data_desc=financial_statements['data_desc']
)

# 5. 如果是港股，提取港股财务分析指标数据包括ROE等
hk_financial_analysis = extract_hk_financial_analysis(stock_type, stock_code)
# 保存到本地

save_extracted_info(
    data_value=hk_financial_analysis['data_value'],
    data_type=hk_financial_analysis['data_type'],
    data_desc=hk_financial_analysis['data_desc']
)
save_collected_data_to_local()

# 6. 提取公司财务摘要信息
financial_summary = extract_financial_summary(stock_type, stock_code)
save_extracted_info(
    data_value=financial_summary['data_value'],
    data_type=financial_summary['data_type'],
    data_desc=financial_summary['data_desc']
)


# 7. 提取行业对比分析数据
field_compare_data = extract_field_compare(stock_type, stock_code)
save_extracted_info(
    data_value=field_compare_data['data_value'],
    data_type=field_compare_data['data_type'],
    data_desc=field_compare_data['data_desc']
)
save_collected_data_to_local()


# 8. 提取股票评级信息
rating_info = extract_rating_info(stock_type, stock_code)
save_extracted_info(
    data_value=rating_info['data_value'],
    data_type=rating_info['data_type'],
    data_desc=rating_info['data_desc']
)



# 9. 提取股票收益和价值预测信息
worth_predict_info = extract_worth_predict(stock_type, stock_code)
save_extracted_info(
    data_value=worth_predict_info['data_value'],
    data_type=worth_predict_info['data_type'],
    data_desc=worth_predict_info['data_desc']
)
save_collected_data_to_local()

# 3. 数据加载

In [3]:
import pandas as pd
import pickle
def load_collected_data_from_local():
    global collected_data_value, collected_data_type, collected_data_desc
    try:
        with open('collected_data_value.pkl', 'rb') as f:
            collected_data_value = pickle.load(f)
        with open('collected_data_type.pkl', 'rb') as f:
            collected_data_type = pickle.load(f)
        with open('collected_data_desc.pkl', 'rb') as f:
            collected_data_desc = pickle.load(f)
        print("数据已从本地加载。")
    except FileNotFoundError:
        print("未找到本地数据文件，请先运行保存操作。")
        
# 信息获取后，数据存储到dict里面
collected_data_value = {}
collected_data_type = {}
collected_data_desc = {}
# 数据加载完毕
load_collected_data_from_local()
# 数据在collected_data_value, collected_data_type, collected_data_desc 

df1 = collected_data_value['资产负债表']
df2 = df1.pivot(index='REPORT_DATE', columns='STD_ITEM_NAME', values='AMOUNT')

# 重置索引，让 REPORT_DATE 变回列（可选）
df2.reset_index(inplace=True)

# 去掉列索引的名字
df2.columns.name = None
# 转换为yyyy-mm-dd格式
df2['REPORT_DATE'] = pd.to_datetime(df2['REPORT_DATE']).dt.strftime('%Y-%m-%d')
collected_data_value['资产负债表'] = df2

df1 = collected_data_value['利润表']
df2 = df1.pivot(index='REPORT_DATE', columns='STD_ITEM_NAME', values='AMOUNT')

# 重置索引，让 REPORT_DATE 变回列（可选）
df2.reset_index(inplace=True)

# 去掉列索引的名字
df2.columns.name = None
# 转换为yyyy-mm-dd格式
df2['REPORT_DATE'] = pd.to_datetime(df2['REPORT_DATE']).dt.strftime('%Y-%m-%d')
collected_data_value['利润表'] = df2

df1 = collected_data_value['现金流量表']
df2 = df1.pivot(index='REPORT_DATE', columns='STD_ITEM_NAME', values='AMOUNT')

# 重置索引，让 REPORT_DATE 变回列（可选）
df2.reset_index(inplace=True)

# 去掉列索引的名字
df2.columns.name = None
# 转换为yyyy-mm-dd格式
df2['REPORT_DATE'] = pd.to_datetime(df2['REPORT_DATE']).dt.strftime('%Y-%m-%d')
collected_data_value['现金流量表'] = df2

df1 = collected_data_value['港股财务分析指标数据']
df1['REPORT_DATE'] = pd.to_datetime(df1['REPORT_DATE']).dt.strftime('%Y-%m-%d')
df2 = df1.loc[:,['REPORT_DATE','ROE_AVG','ROA', 'PER_NETCASH_OPERATE','PER_OI', 'BPS',
       'BASIC_EPS', 'DILUTED_EPS', 'OPERATE_INCOME', 'OPERATE_INCOME_YOY',
       'GROSS_PROFIT', 'GROSS_PROFIT_YOY', 'HOLDER_PROFIT',
       'HOLDER_PROFIT_YOY', 'GROSS_PROFIT_RATIO', 'EPS_TTM',
       'OPERATE_INCOME_QOQ', 'NET_PROFIT_RATIO', 'GROSS_PROFIT_QOQ',
        'HOLDER_PROFIT_QOQ', 'ROE_YEARLY', 'ROIC_YEARLY', 'TAX_EBT',
       'OCF_SALES', 'DEBT_ASSET_RATIO', 'CURRENT_RATIO', 'CURRENTDEBT_DEBT','CURRENCY']]
collected_data_value['港股财务分析指标数据'] = df2

collected_data_size = {}
# 只需要对dataframe类型的数据进行大小计算，其他的认为很小就行，dataframe的返回行列，文本的形式
def gen_collected_data_size():
    global collected_data_value, collected_data_type, collected_data_size
    for key, value in collected_data_value.items():
        if collected_data_type[key] == 'dataframe':
            # 获取DataFrame的行数和列数
            size_info = value.shape
            collected_data_size[key] = "行数为 {}, 列数为 {}".format(size_info[0], size_info[1])
        else:
            # 其他类型认为是小数据
            collected_data_size[key] = {"小的文本数据"}

gen_collected_data_size()

数据已从本地加载。


# 4. 任务拆分

In [4]:
# info_get_agent的输出信息记录，实时记录到文件中保存
info_get_agent_output_log = "info_get_agent_output.log"

# markdown_writer_agent的输出信息记录，实时记录到文件中保存
markdown_writer_agent_output_log = "markdown_writer_agent_output.log"

# 金融研究员agent获取信息的history记录
writer_agent_info_get_history = "writer_agent_info_get_history.log"

# 写作的输出除了写到markdown中，也实时写入一个文件中保存
markdown_writer_output_log = "markdown_writer_output.log"
current_data_write_text_log = 'current_data_write_text_log.log'

# 整体的trace的log
trace_log = "trace.log"
print_log = "print.log"

def write_print_to_log(text,path='print.log'):
    """
    将文本写入日志文件
    :param text: 要写入的文本
    :param path: 日志文件路径
    """
    with open(path, 'a', encoding='utf-8') as f:
        f.write(text + '\n')
        print(text)  # 同时输出到控制台

In [5]:
# RAG的思路
collected_data_value.keys()

# 注意数据里面的对应股票代码，指的是要完成的金融报告的指定公司的股票代码，除了行业价值表现数据和财务经营数据外，其他的数据里面只有要完成的金融报告的指定公司的信息。
collected_data_desc['财务摘要'] = '包含对应股票代码的财务信息的数据摘要分析'
collected_data_desc['股票近期价格和交易信息'] = '包含对应股票的日期、开盘价、收盘价、最高价、最低价、成交量、成交额、振幅、涨跌幅、涨跌额和换手率等信息'


In [ ]:
all_data_chunks = []
def construct_collected_data_chunks(collected_data_value,collected_data_desc):
    """
    构造数据块字符串，包含collected_data_value和collected_data_desc的内容
    :param collected_data_value: 收集到的数据字典
    :param collected_data_desc: 收集到的数据描述字典
    :return: 数据块字符串
    """
    global all_data_chunks
    data_chunks_disc_list = []
    idx = 1
    for key, value in collected_data_value.items():
        desc = collected_data_desc.get(key, "无描述")
        discription = f"数据块{idx}: {key} - {desc}\n"
        data_chunks_disc_list.append(discription)
        # 如果是DataFrame类型的数据，转换为字符串
        if isinstance(value, pd.DataFrame):
            table_intro = f'{key} - {desc}'
            table_str = value.to_csv(sep='|', index=False, header=True)
            result_str = f"{table_intro}\n{table_str}"
        else:
            # 如果是其他类型的数据，直接转换为字符串
            result_str = f"{key} - {desc}\n{str(value)}"
        all_data_chunks.append(result_str)  # 添加数据块内容
        idx += 1
    return "\n".join(data_chunks_disc_list)

data_chunks_disc = construct_collected_data_chunks(collected_data_value, collected_data_desc)

# 任务拆解，把研究任务拆解成多个子任务，每个子任务可以独立完成，按照顺序进行标号，不超过10个子任务

sub_task_plan_template = """
你是一个专业的金融研究员，需要撰写一篇关于 {stock_info} 的金融研究报告。当前的研究任务如下：

任务说明：
------------
{query}
------------

请你根据这个研究任务，按照逻辑顺序拆解为多个子任务。每个子任务应具备以下特征：
- 可以独立完成
- 按照写作或研究的先后顺序排列
- 数量一般不超过 10 个子任务

输出格式要求如下：
- 使用如下格式：
  #1# 子任务说明
  #2# 子任务说明
  ...
- 不使用任何 Markdown 格式
- 不添加任何解释性内容

请开始输出子任务计划。
"""

sub_task_plan_prompt = sub_task_plan_template.format(
    stock_info=input_stock_code,  # 输入的股票代码
    query=task1  # 当前的研究任务
)

sub_task_response = deepseek_v3.invoke(sub_task_plan_prompt)


import re
def generate_subtasks(llm_response):
    # 提取子任务
    subtasks = re.findall(r'#(\d+)#\s*(.+)', llm_response.strip())
    # 按顺序排序并去重
    subtasks = sorted(subtasks, key=lambda x: int(x[0]))
    unique_tasks = {}
    for idx, task in subtasks:
        if task not in unique_tasks:
            unique_tasks[task] = None
    # 返回任务列表
    return list(unique_tasks.keys())

task_list = generate_subtasks(sub_task_response.content)
write_print_to_log("子任务列表："+str(task_list))


In [6]:
task_list = ['收集并整理商汤科技（0020.HK）的基本信息，包括公司背景、上市时间、股票代码、交易所等  ',
 '提取并分析商汤科技的三大会计报表（资产负债表、利润表、现金流量表），整理关键财务数据  ',
 '梳理商汤科技的主营业务、核心竞争力及行业地位，结合行业报告与公开数据  ',
 '计算关键财务比率（如ROE、毛利率、现金流匹配度等），并与同行企业进行横向对比分析  ',
 '横向竞争分析(商汤科技与同行企业财务表现、业务模式及市场策略比较) ',
 '构建估值模型（如DCF、PE等），预测未来财务表现，并模拟关键变量（如成本、汇率）的影响  ',
 '股权结构、公司治理与管理层分析  ',
 '发展战略与未来增长潜力评估',
 '结合财务分析、行业对比及估值结果，提出投资建议（如买入、持有或卖出）  ',
 '识别并列出商汤科技的潜在风险因素（如政策风险、技术风险、市场竞争等），提出风险提醒  ']


## 4.1 RAG——获取相关数据块，数据不足时可以进行搜索，来回答子问题

In [8]:
# 获取最相关的数据chunk块
find_most_relavent_chunks_template = """
    你是一个专业的金融研究员，需要对关于 {stock_info} 的金融财务问题给出非常专业精准的回答，因此你需要先从候选的数据池中选择出最相关的数据块以提供参考信息。
    该问题为：
    -----
    {task}
    -----

    为了回答该问题，你需要从以下数据池中选择最相关的数据块，获取信息作为参考：
    -----
    {data_chunks}
    -----

    请你筛选出回答该问题最有可能用到的数据块编号，最多选择6个数据块，最少选择1个数据块，每个数据块的编号之间用逗号分隔，直接输出编号，不要添加任何其他内容，例如 0,1
    不要解释、分析或者输出任何其他内容，只需要输出数据块标号，你判断需要使用的数据块标号是：
"""

subtask_knowledge_addition_prompt_template = """
    你是一个专业的金融研究员，擅长信息检索和数据分析。
    你需要对关于 {task} 的问题给出非常专业精准的回答，目前你搜索到的相关信息包括：
    -----
    {used_data_chunk}
    -----
    
    请你判断，这些信息是否足够你回答该问题，还是需要更多的信息，请你以json格式输出，格式如下：
    {{
        "is_enough": "是" 或 "否",
        "need_info": "如果需要更多信息，请用一句话说明具体需要什么信息；如果不需要更多信息，请写无"
    }}
"""


# 用于生成报告正文内容的模板
subtask_solution_prompt_template = """
你是一个专业的金融研究员，正在撰写关于 {stock_info} 的公司研究报告。

当前章节需要涵盖的内容是：
-----
{task}
-----

你需要参考以下数据和信息：
-----
{used_data_chunk}
-----

请根据以上信息，撰写该章节的正式报告内容。要求如下：

1. 表述专业、逻辑清晰、内容翔实，适合用于正式研究报告；
2. 若涉及数据，应保证数据和参考信息里面一致，不能编造；
3. 若涉及表格，应使用Markdown格式输出表格，表格内容要清晰、易读，可以只呈现关键数据，例如按照年份进行采样或者汇总输出，但是表格内容要完整不能出现省略号；
4. 若涉及分析，应给出结论性判断（如趋势、对比、风险、机会等）；
5. 不要写“我将如何分析”，而是直接输出分析结论；
6. 不要写“步骤一、步骤二”；
7. 使用中文，语句通顺，格式规范。

请直接以正式报告内容的形式输出该章节内容。
"""

In [9]:
async def search_addition_data(query):
    answer = await financial_search(query)
    return answer

In [ ]:
task_solution_list = []
for i in range(len(task_list)): # 依次完成任务
    task = task_list[i]  # 当前任务
    write_print_to_log(f"\n\n--------------------开始处理任务 {i+1}: {task} --------------------\n")
    retry = 0
    finish_flag = 0
    while retry<3 and finish_flag<=0:
        # 搜集和任务相关的信息，可以取出最相关的6个chunk
        find_most_relavent_chunks_prompt = find_most_relavent_chunks_template.format(
            stock_info=input_stock_code,  # 输入的股票代码
            task=task,  # 当前的任务
            data_chunks=data_chunks_disc  # 数据块描述字符串
        )
        find_chunk_response = qwen_plus.invoke(find_most_relavent_chunks_prompt,enable_thinking=False)
        sleep(1)
        most_relevant_chunks = find_chunk_response.content.strip()  # 最相关的数据块编号
        # 解析chunk编号
        most_relevant_chunks_list = most_relevant_chunks.split(',')  # 分割成列表
        # 检查数据格式是否准确
        most_relevant_chunks_list = [chunk.strip() for chunk in most_relevant_chunks_list if chunk.strip().isdigit()]
        if len(most_relevant_chunks_list) == 0:
            write_print_to_log(f"没有找到相关的数据块")
            most_relevant_chunks_list = [] # 没有数据就不用了，搜索！
        write_print_to_log(f"最相关的数据块编号: {most_relevant_chunks_list}")
        # trace_log输出
        with open(trace_log, 'a', encoding='utf-8') as f:
            f.write(f"Most relevant chunks: {most_relevant_chunks}\n")
        finish_flag = 1 # 有数据了
        # 构建数据块字符串
        used_data_chunk = []
        for k in range(len(most_relevant_chunks_list)):
            chunk_index = int(most_relevant_chunks_list[k])
            if chunk_index < len(all_data_chunks):
                used_data_chunk.append(all_data_chunks[chunk_index])
        
        used_data_chunk_str = '\n\n'.join(used_data_chunk)

        # 判断是否需要更多的信息
        subtask_knowledge_addition_prompt = subtask_knowledge_addition_prompt_template.format(
            task=task,  # 当前的任务
            used_data_chunk=used_data_chunk_str  # 使用的数据块字符串
        )
        subtask_knowledge_addition_response = qwen_plus.invoke(subtask_knowledge_addition_prompt,enable_thinking=False)
        sleep(1)
        knowledge_addition_json_str = subtask_knowledge_addition_response.content.strip()
        write_print_to_log(f"Knowledge addition json: {knowledge_addition_json_str}")
        # trace_log输出
        with open(trace_log, 'a', encoding='utf-8') as f:
            f.write(f"Knowledge addition json: {knowledge_addition_json_str}\n")
        try:
            knowledge_addition_json = json.loads(knowledge_addition_json_str)
            is_enough = knowledge_addition_json.get('is_enough', '否')
            need_info = knowledge_addition_json.get('need_info', '无')
        except json.JSONDecodeError:
            is_enough = '否'
            need_info = '无'
        if is_enough == '否' and need_info != '无':
            write_print_to_log(f"需要更多的信息: {need_info}")
            # 调用搜索工具，获取更多的信息
            additional_info = await search_addition_data(need_info)
            write_print_to_log(f"搜索到的额外信息: {additional_info}")
            used_data_chunk_str += f"\n\n额外搜索信息:\n{additional_info}"
            
        # 生成任务解决方案
        subtask_solution_prompt = subtask_solution_prompt_template.format(
            stock_info=input_stock_code,  # 输入的股票代码
            task=task,  # 当前的任务
            used_data_chunk=used_data_chunk_str  # 使用的数据块字符串
        )

        subtask_solution_response = qwen_plus.invoke(subtask_solution_prompt)
        sleep(1)
        subtask_solution = subtask_solution_response.content
        task_solution_list.append(subtask_solution)  # 存储任务解决方案
        # trace_log输出
        with open(trace_log, 'a', encoding='utf-8') as f:
            f.write(f"Subtask solution: {subtask_solution}\n")
        write_print_to_log('\n\n--------------------子任务解决方案--------------------\n')
        write_print_to_log(f"Subtask solution:\n {subtask_solution}")

# 数据保存，保存任务解决方案到pickle
import pickle
with open('task1_solution_list_0924.pkl', 'wb') as f:
    pickle.dump(task_solution_list, f)


# 4.2 数据分析、图表生成，对子任务内容重构

In [ ]:

# subtask_solution_prompt_template = """
# 你是一个专业的金融研究员，正在撰写关于 {stock_info} 的公司研究报告。

# 当前章节需要涵盖的内容是：
# -----
# {task}
# -----

# 你需要参考以下数据和信息：
# -----
# {used_data_chunk}
# -----

# 请根据以上信息，撰写该章节的正式报告内容。要求如下：

# 1. 表述专业、逻辑清晰、内容翔实，适合用于正式研究报告；
# 2. 若涉及数据，应保证数据和参考信息里面一致，不能编造；
# 3. 若涉及表格，应使用Markdown格式输出表格，表格内容要清晰、易读，可以只呈现关键数据，例如按照年份进行采样或者汇总输出，但是表格内容要完整不能出现省略号；
# 4. 若涉及分析，应给出结论性判断（如趋势、对比、风险、机会等）；
# 5. 不要写“我将如何分析”，而是直接输出分析结论；
# 6. 不要写“步骤一、步骤二”；
# 7. 使用中文，语句通顺，格式规范。

# 请直接以正式报告内容的形式输出该章节内容。
# """

# # 输入也包含task和subtask
# # 生成占位符，把数据拿出去，要画图的，整理出一个符合的字段格式
# # 包括是画图还是表格，还是要做预测或者计算，有一个task字段
# # if_calculate表示是否需要计算
# # pic_type表示是画图还是画表格，默认画图使用matplotlib和seaborn，画表格使用html
# # 然后也需要一个描述性的信息，同时原文里面也要有对应的占位符号，就是需要提到如【图X】或者【表X】这样的占位符号，后续填充替换



#我在做研报生成的agent，subtask_solution_prompt_template是负责写一个章节的，这个章节的内容里面有一些数据可能适合根据章节内容来做一些深入的分析，或者是配上一些比较好看的图表、表格，为了便于后面我接上图表和表格绘制的工具，是不是对章节的内容重新提取一下比较好，如果是适合直接文本输出的，就直接保留原文包括原始的格式，如果有一个片段很适合进行细化的数据分析、绘制图表或者表格来辅助展示，把这个片段使用xml格式的标记进行标注，<figure>和</figure>标记这个片段的开始和结尾，中间呢，首先包括<type></type>这个标注，中间注明是"数据分析"、"绘制图表"还是"插入表格"，注意绘制图表会使用matplotlib绘制折线图、趋势图、柱状图等，"插入表格"会生成html格式代码插入表格，<type></type>后面跟着<src></src>这个标注，这个里面放原始的这个片段的所有内容，防止万一分析或者画图画表失败了内容丢失，<src></src>这个标注后再有一个<illustration></illustration>，这里面放的是，假如画图画表准确，得出了想要的东西，在研报里面要如何描述图表的内容，因为暂时还不知道具体这个图/表的序号是多少，所以在引述时，使用图[figure]或者表[table]这样来指代这个图或者是表，最后，需要一个<generation></generation>这个tag，这个是会作为数据分析或者图表生成的工具的输入，这里面要清晰的说清楚要分析什么数据（例如同比、环比或者计算roe）、或者是绘制什么类型的图和表，同时需要把所有需要用到的数据清晰的列在这里（例如表格格式的数据），这些会被用于图表生成数据的输入。我的章节生成的prompt如下供你参考，你帮我写一下完成这个操作的prompt吧，名字叫figure_gen_prompt_template吧，


In [10]:
figure_generation_prompt_template = """你是一个专业的金融研究员，正在撰写关于 {stock_info} 的公司研究报告。

当前章节需要涵盖的内容是：
-----
{subtask}
-----

当前章节草稿：
-----
{section_draft}
-----

请根据以上信息，撰写该章节的正式报告内容。要求如下：

1. 表述专业、逻辑清晰、内容翔实，适合用于正式研究报告；
2. 若涉及数据，应保证数据和草稿里面一致，不能编造；
3. 在内容中，如果发现某部分内容特别适合通过数据分析、绘制图表或插入表格来辅助说明，以便于后续调用工具丰富内容，需要对该部分内容进行XML格式的标记；
4. 注意，请以单次数据分析、单次绘图或者单次插入表格为一个标记单元，如果有一处适合插入多个图表或者多个表格，请分开多次标记；
5. 请使用以下XML格式进行标记：
   
   - 对于适合进行详细数据分析的部分：
     <figure>
       <type>数据分析</type>
       <src>{{原始章节草稿内容}}</src>
       <generation>具体的数据分析指令（例如要分析什么数据、如何分析），以及要分析的目标（例如计算同比、环比等）。</generation>
       <illustration>这里描述预期数据分析任务正常完成时如何解释数据和数据分析的结论得出最终的结论。</illustration>
     </figure>

   - 对于适合绘制图表的部分：
     <figure>
       <type>绘制图表</type>
       <src>{{原始章节草稿内容}}</src>
       <generation>具体的绘图要求指令，包括图表类型（如折线图、柱状图等）及所需的原始数据(画图指令中要提到要求直接生成1个html格式的图表)。</generation>
       <illustration>图[figure]展示了...（描述预期的图表内容）。</illustration>
     </figure>

   - 对于适合插入表格的部分：
     <figure>
       <type>插入表格</type>
       <src>{{原始章节草稿内容}}</src>
       <generation>具体的表格绘制要求指令，以及具体的原始数据(画图指令中要提到要求直接生成1个html格式的图表)</generation>
       <illustration>表[table]列出了...（描述预期的图表内容）。</illustration>
     </figure>
4. 注意在做XML标记时，对于<illustration>字段，因为暂时还不知道具体这个图/表的序号是多少，所以在描述时，使用图[figure]或者表[table]这样来指代这个图或者是表
5. 如果内容不适合上述任何一种情况，请保持原文本格式不变。
6. 使用中文，语句通顺，格式规范。

请直接以正式报告内容的形式输出该章节内容，并确保所有标记遵循指定的格式。"""

In [11]:
import re
from turtle import begin_fill
from typing import List, Dict, Tuple

def parse_report_with_figures(full_content):
    """
    解析包含 <figure>...</figure> 标记的研报内容。

    返回：
        - cleaned_text: 原文中的 <figure> 被替换为 [FIGURE_PLACEHOLDER_N] 占位符
        - figure_list: 所有 figure 片段的结构化解析结果列表，包含完整 generation 内容
    """
    figure_list = []
    cleaned_text = full_content
    matches = list(re.finditer(r"<figure>\s*(.*?)\s*</figure>", full_content, re.DOTALL))

    if not matches:
        return full_content.strip(), []

    for idx, match in enumerate(matches):
        inner_content = match.group(1)

        # 手动提取各字段（避免 XML 解析问题）
        def extract_tag(tag_name: str) -> str:
            pattern = rf"<{tag_name}>(.*?)</{tag_name}>"
            result = re.search(pattern, inner_content, re.DOTALL)
            return result.group(1).strip() if result else ""

        type_text = extract_tag("type")
        src_text = extract_tag("src")
        illustration_text = extract_tag("illustration")
        generation_text = extract_tag("generation")  # 完整保留 generation 内容

        if not type_text or type_text not in ["数据分析", "绘制图表", "插入表格"]:
            print(f"警告：跳过无效的 figure（缺少 type 或 type 不合法）:\n{inner_content}")
            continue

        # 构建 figure 数据
        figure_data = {
            "type": type_text,
            "src": src_text,
            "illustration": illustration_text,
            "generation": generation_text  # 完整保留原始内容，包括 HTML 和换行
        }
        figure_list.append(figure_data)

        # 用占位符替换原始 <figure> 块，便于后续插入图表
        placeholder = f"[FIGURE_PLACEHOLDER_{idx}]"
        cleaned_text = cleaned_text.replace(match.group(0), placeholder)

    return cleaned_text.strip(), figure_list

In [ ]:
# with open('figure_generation_response.xml', 'r', encoding='utf-8') as f:
#     section_content = f.read()
# cleaned_section_text, extracted_figures = parse_report_with_figures(section_content)


# gen_table_task = ""
# if True:
#     for ti in range(len(extracted_figures)):
#         if ti!=3:
#             continue
#         figure = extracted_figures[ti]
#         if figure['type'] == '插入表格':
#             gen_table_task += f"任务要求：请根据以下内容生成一个HTML格式的表格。\n"
#             gen_table_task += f"预期表格描述：{figure['illustration']}\n"
#             gen_table_task += f"原始数据：\n{figure['src']}\n\n"
#             gen_table_task += f"生成表格的要求/提示或部分参考代码:\n{figure['generation']}\n\n"
#         elif figure['type'] == '绘制图表':
#             gen_table_task += f"任务要求：请根据以下内容使用matplotlib和seaborn生成一个图表。\n"
#             gen_table_task += f"预期图表描述：{figure['illustration']}\n"
#             gen_table_task += f"原始数据：\n{figure['src']}\n\n"
#             gen_table_task += f"生成图表的要求/提示或部分参考代码:\n{figure['generation']}\n\n"
#         elif figure['type'] == '数据分析':
#             gen_table_task += f"任务要求：请根据以下内容进行数据分析。\n"
#             gen_table_task += f"预期分析描述：{figure['illustration']}\n"
#             gen_table_task += f"原始数据：\n{figure['src']}\n\n"
#             gen_table_task += f"数据分析的要求/提示或部分参考代码:\n{figure['generation']}\n\n"

In [ ]:

# figure_generation_prompt = figure_generation_prompt_template.format(
#     stock_info=input_stock_code,  # 输入的股票代码
#     subtask=task_list[1],  # 当前的任务
#     section_draft=task_solution_list[0]  # 当前章节草稿
# )
# figure_generation_response = qwen_plus.invoke(figure_generation_prompt)

# # 存一下这个字符串，便于后面调试
# with open('figure_generation_response.xml', 'w', encoding='utf-8') as f:
#     f.write(figure_generation_response.content)

# with open('figure_generation_response.xml', 'r', encoding='utf-8') as f:
#     section_content = f.read()

# 可以先判断是否需要先数据分析再画图，还是直接画图就可以

# 这样所有的都可以统一走数据分析的接口了

# 数据分析完了之后再判断是否需要画图，还是描述性的形式

In [12]:
from python_data_analyse import *
async def gen_table(gen_table_task,table_index=0):
    current_path = os.getcwd()
    os.makedirs(f"{current_path}/gen_tables/", exist_ok=True)
    max_attempts = 3
    for current_attempt in range(max_attempts):
        if current_attempt==0:
            html_table_gen_prompt = html_table_gen_template.format(task=gen_table_task)
        else:
            html_table_gen_prompt = html_table_review_gen_template.format(task=gen_table_task,
                                                                        pre_code=html_table_response.content,
                                                                        vlm_feedback=vlm_feedback)
        print(html_table_gen_prompt)
        html_table_response = qwen_plus.invoke(html_table_gen_prompt)
        with open(f"{current_path}/gen_tables/table_{table_index}_{current_attempt}.html", "w") as f:
            f.write(html_table_response.content)

        # --- 使用 ---
        try:
            await screenshot_with_playwright_async(f"{current_path}/gen_tables/table_{table_index}_{current_attempt}.html", 'div', f"{current_path}/gen_tables/table_{table_index}_{current_attempt}.png")
        except Exception as e:
            print(f"Error during screenshot: {e}")
            vlm_feedback = ""
            continue
        # 质检
        vlm_feedback = vlm_check_function(f"{current_path}/gen_tables/table_{table_index}_{current_attempt}.png",task=gen_table_task)

        if "PASS" in vlm_feedback:
            print("VLM quality check passed.")
            break
        else:
            print("VLM quality check failed.")
        print(vlm_feedback)
    if 'PASS' in vlm_feedback: # 如果通过了质检，保存
        return f"{current_path}/gen_tables/table_{table_index}_{current_attempt}.png"
    else: # 否则质量太差，还不如原始的，转markdown
        return None

In [13]:
from python_data_analyse import *

async def gen_figure(gen_figure_task, figure_index=0):
    current_path = os.getcwd()
    os.makedirs(f"{current_path}/gen_figures/", exist_ok=True)
    max_attempts = 3
    for current_attempt in range(max_attempts):
        if current_attempt==0:
            html_pic_gen_prompt = html_pic_gen_template.format(task=gen_figure_task)
        else:
            html_pic_gen_prompt = html_pic_review_gen_template.format(task=gen_figure_task,
                                                                       pre_code=html_table_response.content,
                                                                       vlm_feedback=vlm_feedback)
        print(html_pic_gen_prompt)
        html_table_response = qwen_plus.invoke(html_pic_gen_prompt)
        with open(f"{current_path}/gen_figures/figure_{figure_index}_{current_attempt}.html", "w") as f:
            f.write(html_table_response.content)
        # step2 截图
        try:
            await screenshot_with_playwright_async(f"{current_path}/gen_figures/figure_{figure_index}_{current_attempt}.html", 'div', f"{current_path}/gen_figures/figure_{figure_index}_{current_attempt}.png")
        except Exception as e:
            print(f"截图失败: {e}")
            vlm_feedback = ""
            continue
        # step3  质检
        vlm_feedback = vlm_check_function(f"{current_path}/gen_figures/figure_{figure_index}_{current_attempt}.png", task1)

        if "PASS" in vlm_feedback:
            print("VLM quality check passed.")
            break
        else:
            print("VLM quality check failed.")

        print(vlm_feedback)
    if 'PASS' in vlm_feedback: # 如果通过了质检，保存
        return f"{current_path}/gen_figures/figure_{figure_index}_{current_attempt}.png"
    else: # 否则质量太差，还不如原始的
        return None

In [23]:
from python_data_analyse import *
def data_anylyse(data_analysis_task):
    max_attempts = 3
    current_attempt = 0
    print_info = None
    while current_attempt < max_attempts:
        with Sandbox.create(api_key=E2B_API_KEY) as code_interpreter:
            task_prompt = data_anylyse_prompt_template.format(task=data_analysis_task)

            if current_attempt==0:
                code_results,python_code = chat_with_llm(
                    code_interpreter,
                    task_prompt
                )
            else:
                code_results,python_code = chat_with_llm(
                    code_interpreter,
                    task_prompt,
                    pre_code=python_code,
                    pre_error=code_results[1].traceback
                )
            if code_results is not None and code_results[0]==0:  # 执行成功
                print_info = code_results[1]
                break
            current_attempt += 1
    return print_info

In [ ]:
# from python_data_analyse import *
# if True:
#     max_attempts = 3
#     current_attempt = 0
#     while current_attempt < max_attempts:
#         with Sandbox.create(api_key=E2B_API_KEY) as code_interpreter:
#             task_prompt = data_anylyse_prompt_template.format(task=gen_table_task)

#             if current_attempt==0:
#                 code_results,python_code = chat_with_llm(
#                     code_interpreter,
#                     task_prompt
#                 )
#             else:
#                 code_results,python_code = chat_with_llm(
#                     code_interpreter,
#                     task_prompt,
#                     pre_code=python_code,
#                     pre_error=code_results[1].traceback
#                 )
#             if code_results[0]==0:  # 执行成功
#                 print_info = code_results[1]
#                 break
#             current_attempt += 1

In [15]:
vlm_gen_table_description_template = """
You are a professional financial researcher preparing content for an investment research report. 
Based on the following data, generate a clear, insightful, and professionally written description of the table.

The data used to generate the table is as follows:
{data_content}

Your description should:
1. Be written in the same language as the input data.
2. Refer to the table as "表[table]" since the exact number is unknown.
3. Highlight key trends, notable changes, or significant observations (e.g., growth, decline, peaks, outliers).
4. Emphasize comparative insights (e.g., year-on-year changes, regional differences, sector performance).
5. Use formal and concise language suitable for a professional research report.
6. Avoid repeating raw data; instead, interpret what the data implies.

Example (if data shows revenue growth):
"表[table]展示了各业务板块近三年的收入表现。其中，云计算业务持续高速增长，2023年同比增幅达35%，成为主要增长驱动力；而传统软件业务增速放缓，呈现平稳态势。"

Now, generate the description:
"""

vlm_gen_figure_description_template = """
You are a professional financial researcher preparing content for an investment research report. 
Based on the following data, generate a clear, insightful, and professionally written description of the figure.

The data used to generate the figure is as follows:
{data_content}

Your description should:
1. Be written in the same language as the input data.
2. Refer to the figure as "图[figure]" since the exact number is unknown.
3. Describe the overall trend (e.g., upward, fluctuating, cyclical).
4. Identify key turning points, peaks, troughs, or structural changes.
5. Highlight any notable patterns, correlations, or anomalies.
6. Use formal, precise, and objective language suitable for a research report.
7. Avoid merely restating data; focus on interpretation and implications.

Example (if chart shows stock performance):
"图[figure]显示标的股价在过去一年中呈现震荡上行趋势，尤其在Q4受政策利好推动，涨幅显著。期间虽有短期回调，但整体支撑位稳固，市场信心逐步恢复。"

Now, generate the description:
"""

data_analyse_result_description_template = """
You are a professional financial researcher writing a concise and insightful commentary for an investment research report. 
Your output will be a single, well-structured paragraph that interprets the data and analysis results in context.

---
Raw Data:
{data_content}

Analysis Results:
{analysis_results}
---

Instructions:
1. Write in the **same language as the input data** (e.g., Chinese if data is in Chinese).
2. Synthesize the data and results into **one clear, fluent paragraph** (3–5 sentences).
3. Highlight key trends, notable changes, or implications — don’t just repeat numbers.
4. Use professional, objective tone suitable for a research report.
5. Focus on **what the data means**, not just what it shows.
6. Do **not** use bullet points, headings, or markdown formatting.
7. Keep it concise and impactful — this is a supporting commentary, not a standalone analysis.

Example output (if data shows growth):
"公司收入从2021年的120亿元增长至2023年的180亿元，三年复合增速达22.4%，2023年同比上升24%。2024年前三个季度实现收入150亿元，已接近2023年全年的83%，全年业绩确定性较高，增长动能依然充足。"

Now, generate the description:
"""

def vlm_description_function(image_path,prompt_template, data_content):
    image_data = image_to_base64(image_path)
    vlm_response = client.chat.completions.create(
        model=qwen_vl_model,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_template.format(data_content=data_content)},
                    {"type": "image_url", "image_url": {"url": image_data}},  # 使用 base64 数据
                ],
            }
        ],
        temperature=0
    )
    vlm_feedback = vlm_response.choices[0].message
    return vlm_feedback

In [ ]:
# import pickle
# with open('task1_solution_list_0924.pkl', 'rb') as f:
#     task_solution_list = pickle.load(f)

In [ ]:
# task_list = ['收集并整理商汤科技（0020.HK）的基本信息，包括公司背景、上市时间、股票代码、交易所等  ',
#  '提取并分析商汤科技的三大会计报表（资产负债表、利润表、现金流量表），整理关键财务数据  ',
#  '梳理商汤科技的主营业务、核心竞争力及行业地位，结合行业报告与公开数据  ',
#  '计算关键财务比率（如ROE、毛利率、现金流匹配度等），并与同行企业进行横向对比分析  ',
#  '横向竞争分析(商汤科技与同行企业财务表现、业务模式及市场策略比较) ',
#  '构建估值模型（如DCF、PE等），预测未来财务表现，并模拟关键变量（如成本、汇率）的影响  ',
#  '股权结构、公司治理与管理层分析  ',
#  '发展战略与未来增长潜力评估',
#  '结合财务分析、行业对比及估值结果，提出投资建议（如买入、持有或卖出）  ',
#  '识别并列出商汤科技的潜在风险因素（如政策风险、技术风险、市场竞争等），提出风险提醒  ']


In [ ]:
# from threading import Lock

# lock = Lock()
# global_figure_index = 0

# def get_unique_figure_index():
#     global global_figure_index
#     with lock:
#         current_index = global_figure_index
#         global_figure_index += 1
#     return current_index

In [ ]:
# async def process_subtask(i,task_solution_list,task_list):
#     write_print_to_log(f"\n\n--------------------开始处理画图任务 {i+1}: {task_list[i]} --------------------\n")
#     figure_generation_prompt = figure_generation_prompt_template.format(
#         stock_info=input_stock_code,  # 输入的股票代码
#         subtask=task_list[i],  # 当前的任务
#         section_draft=task_solution_list[i]  # 当前章节草稿
#     )
#     figure_generation_response = qwen_plus.invoke(figure_generation_prompt)
#     # 解析章节内容
#     cleaned_section_text, extracted_figures = parse_report_with_figures(figure_generation_response.content)
#     # extracted_figures是列表
#     gen_table_task = ""
#     extracted_result_list = []
#     for ti in range(len(extracted_figures)):
#         figure = extracted_figures[ti]
#         if figure['type'] == '插入表格':
#             gen_table_task += f"任务要求：请根据以下内容生成一个HTML格式的表格。\n"
#             gen_table_task += f"预期表格描述：{figure['illustration']}\n"
#             gen_table_task += f"原始数据：\n{figure['src']}\n\n"
#             gen_table_task += f"生成表格的要求/提示或部分参考代码:\n{figure['generation']}\n\n"
#             table_path = await gen_table(gen_table_task,table_index=get_unique_figure_index())
#             if table_path is not None:
#                 # 生成表的描述信息，注意使用表[table]占位符
#                 table_descrition_response = vlm_description_function(table_path,vlm_gen_table_description_template,
#                                                                      figure['illustration']+'\n'+figure['src']+'\n'+figure['generation'])
#                 extracted_result_list.append({
#                     "type": "table",
#                     "path": table_path,
#                     "illustration": figure['illustration'],
#                     "src": figure['src'],
#                     "generation": figure['generation'],
#                     "description": table_descrition_response.content
#                 })
#             else:
#                 extracted_result_list.append({
#                     "type": "table",
#                     "path": None,
#                     "illustration": figure['illustration'],
#                     "src": figure['src'],
#                     "generation": figure['generation']
#                 })
#         elif figure['type'] == '绘制图表':
#             gen_table_task += f"任务要求：请根据以下内容使用matplotlib和seaborn生成一个图表。\n"
#             gen_table_task += f"预期图表描述：{figure['illustration']}\n"
#             gen_table_task += f"原始数据：\n{figure['src']}\n\n"
#             gen_table_task += f"生成图表的要求/提示或部分参考代码:\n{figure['generation']}\n\n"
#             figure_path = await gen_figure(gen_table_task,figure_index=get_unique_figure_index())
#             if figure_path is not None:
#                 # 生成图的描述信息，注意使用图[table]占位符
#                 figure_descrition_response = vlm_description_function(figure_path,vlm_gen_figure_description_template,
#                                                                       figure['illustration']+'\n'+figure['src']+'\n'+figure['generation'])
#                 extracted_result_list.append({
#                     "type": "figure",
#                     "path": figure_path,
#                     "illustration": figure['illustration'],
#                     "src": figure['src'],
#                     "generation": figure['generation'],
#                     "description": figure_descrition_response.content
#                 })
#             else:
#                 extracted_result_list.append({
#                     "type": "figure",
#                     "path": None,
#                     "illustration": figure['illustration'],
#                     "src": figure['src'],
#                     "generation": figure['generation']
#                 })
#         elif figure['type'] == '数据分析':
#             gen_table_task += f"任务要求：请根据以下内容进行数据分析。\n"
#             gen_table_task += f"预期分析描述：{figure['illustration']}\n"
#             gen_table_task += f"原始数据：\n{figure['src']}\n\n"
#             gen_table_task += f"数据分析的要求/提示或部分参考代码:\n{figure['generation']}\n\n"
#             analysis_result = data_anylyse(gen_table_task)
#             if analysis_result is not None:
#                 # 生成分析的描述信息
#                 analysis_descrition_prompt = data_analyse_result_description_template.format(
#                     data_content=figure['illustration'] + '\n' + figure['src'] + '\n' + figure['generation'],
#                     analysis_results=analysis_result
#                 )
#                 analysis_descrition_response = qwen_plus.invoke(analysis_descrition_prompt)
#                 extracted_result_list.append({
#                     "type": "analysis",
#                     "result": analysis_result,
#                     "illustration": figure['illustration'],
#                     "src": figure['src'],
#                     "generation": figure['generation'],
#                     "description": analysis_descrition_response.content
#                 })
#             else:
#                 extracted_result_list.append({
#                     "type": "analysis",
#                     "result": None,
#                     "illustration": figure['illustration'],
#                     "src": figure['src'],
#                     "generation": figure['generation']
#                 })
#     # 替换cleaned_section_text里面的内容[FIGURE_PLACEHOLDER_I]，替换为对应的图或者表或者分析结论，为此，在给定图/表的情况下，需要生成图/表的描述，以及分析的结论
#     final_section_text = cleaned_section_text
#     for idx in range(len(extracted_result_list)):
#         result = extracted_result_list[idx]
#         placeholder = f"[FIGURE_PLACEHOLDER_{idx}]"
#         if result['type'] == 'table':
#             if result['path'] is not None:
#                 replacement_text = f"![表[table]]({result['path']})\n\n{result['description']}"
#             else:
#                 replacement_text = f"{result['illustration']}\n\n{result['src']}\n\n{result['generation']}"
#         elif result['type'] == 'figure':
#             if result['path'] is not None:
#                 replacement_text = f"![图[figure]]({result['path']})\n\n{result['description']}"
#             else:
#                 replacement_text = f"{result['illustration']}\n\n{result['src']}\n\n{result['generation']}"
#         elif result['type'] == 'analysis':
#             if result['result'] is not None:
#                 replacement_text = f"{result['description']}"
#             else:
#                 replacement_text = f"{result['illustration']}\n\n{result['src']}\n\n{result['generation']}"
#         final_section_text = final_section_text.replace(placeholder, replacement_text)
#     # 保存最终的章节内容
#     return i,final_section_text

In [ ]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import asyncio

# # 创建一个信号量，最多允许 2 个任务同时运行
# semaphore = asyncio.Semaphore(2)

# async def process_subtask_with_limit(i, task_solution_list, task_list):
#     async with semaphore:  # 进入临界区（最多2个并发）
#         result = await process_subtask(i, task_solution_list, task_list)
#         return i, result  # 返回索引和结果

# # 初始化结果列表
# final_subtask_list = [""] * len(task_solution_list)

# # 创建所有任务
# tasks = [
#     process_subtask_with_limit(i, task_solution_list, task_list)
#     for i in range(len(task_solution_list))
# ]

# # 并发执行所有任务，但最多只有 2 个同时运行
# results = await asyncio.gather(*tasks, return_exceptions=True)

# # 处理结果
# for res in results:
#     if isinstance(res, Exception):
#         print(f"任务执行出错: {res}")
#         break
#     i, result = res
#     final_subtask_list[i] = result
#     write_print_to_log(f"\n\n-------------------- 完成处理画图任务 {i+1} --------------------\n")

# # 保存结果
# with open('final_subtask_list_0924.pkl', 'wb') as f:
#     pickle.dump(final_subtask_list, f)

In [ ]:
# 生成图表，正常情况下是一一对应的
global_figure_index = 0 # 可能是图，可能是表，可能是分析
final_subtask_list = []
for i in range(4,len(task_solution_list)):
    write_print_to_log(f"\n\n--------------------开始处理画图任务 {i+1}: {task_list[i]} --------------------\n")
    figure_generation_prompt = figure_generation_prompt_template.format(
        stock_info=input_stock_code,  # 输入的股票代码
        subtask=task_list[i],  # 当前的任务
        section_draft=task_solution_list[i]  # 当前章节草稿
    )
    figure_generation_response = qwen_plus.invoke(figure_generation_prompt)
    # 解析章节内容
    cleaned_section_text, extracted_figures = parse_report_with_figures(figure_generation_response.content)
    # extracted_figures是列表
    gen_table_task = ""
    extracted_result_list = []
    for ti in range(len(extracted_figures)):
        figure = extracted_figures[ti]
        if figure['type'] == '插入表格':
            gen_table_task += f"任务要求：请根据以下内容生成一个HTML格式的表格。\n"
            gen_table_task += f"预期表格描述：{figure['illustration']}\n"
            gen_table_task += f"原始数据：\n{figure['src']}\n\n"
            gen_table_task += f"生成表格的要求/提示或部分参考代码:\n{figure['generation']}\n\n"
            table_path = await gen_table(gen_table_task,table_index=global_figure_index)
            if table_path is not None:
                global_figure_index += 1
                # 生成表的描述信息，注意使用表[table]占位符
                table_descrition_response = vlm_description_function(table_path,vlm_gen_table_description_template,
                                                                     figure['illustration']+'\n'+figure['src']+'\n'+figure['generation'])
                extracted_result_list.append({
                    "type": "table",
                    "path": table_path,
                    "illustration": figure['illustration'],
                    "src": figure['src'],
                    "generation": figure['generation'],
                    "description": table_descrition_response.content
                })
            else:
                extracted_result_list.append({
                    "type": "table",
                    "path": None,
                    "illustration": figure['illustration'],
                    "src": figure['src'],
                    "generation": figure['generation']
                })
        elif figure['type'] == '绘制图表':
            gen_table_task += f"任务要求：请根据以下内容使用matplotlib和seaborn生成一个图表。\n"
            gen_table_task += f"预期图表描述：{figure['illustration']}\n"
            gen_table_task += f"原始数据：\n{figure['src']}\n\n"
            gen_table_task += f"生成图表的要求/提示或部分参考代码:\n{figure['generation']}\n\n"
            figure_path = await gen_figure(gen_table_task,figure_index=global_figure_index)
            if figure_path is not None:
                global_figure_index += 1
                # 生成图的描述信息，注意使用图[table]占位符
                figure_descrition_response = vlm_description_function(figure_path,vlm_gen_figure_description_template,
                                                                      figure['illustration']+'\n'+figure['src']+'\n'+figure['generation'])
                extracted_result_list.append({
                    "type": "figure",
                    "path": figure_path,
                    "illustration": figure['illustration'],
                    "src": figure['src'],
                    "generation": figure['generation'],
                    "description": figure_descrition_response.content
                })
            else:
                extracted_result_list.append({
                    "type": "figure",
                    "path": None,
                    "illustration": figure['illustration'],
                    "src": figure['src'],
                    "generation": figure['generation']
                })
        elif figure['type'] == '数据分析':
            gen_table_task += f"任务要求：请根据以下内容进行数据分析。\n"
            gen_table_task += f"预期分析描述：{figure['illustration']}\n"
            gen_table_task += f"原始数据：\n{figure['src']}\n\n"
            gen_table_task += f"数据分析的要求/提示或部分参考代码:\n{figure['generation']}\n\n"
            analysis_result = data_anylyse(gen_table_task)
            if analysis_result is not None:
                # 生成分析的描述信息
                analysis_descrition_prompt = data_analyse_result_description_template.format(
                    data_content=figure['illustration'] + '\n' + figure['src'] + '\n' + figure['generation'],
                    analysis_results=analysis_result
                )
                analysis_descrition_response = qwen_plus.invoke(analysis_descrition_prompt)
                extracted_result_list.append({
                    "type": "analysis",
                    "result": analysis_result,
                    "illustration": figure['illustration'],
                    "src": figure['src'],
                    "generation": figure['generation'],
                    "description": analysis_descrition_response.content
                })
            else:
                extracted_result_list.append({
                    "type": "analysis",
                    "result": None,
                    "illustration": figure['illustration'],
                    "src": figure['src'],
                    "generation": figure['generation']
                })
    # 替换cleaned_section_text里面的内容[FIGURE_PLACEHOLDER_I]，替换为对应的图或者表或者分析结论，为此，在给定图/表的情况下，需要生成图/表的描述，以及分析的结论
    final_section_text = cleaned_section_text
    for idx in range(len(extracted_result_list)):
        result = extracted_result_list[idx]
        placeholder = f"[FIGURE_PLACEHOLDER_{idx}]"
        if result['type'] == 'table':
            if result['path'] is not None:
                replacement_text = f"![表[table]]({result['path']})\n\n{result['description']}"
            else:
                replacement_text = f"{result['illustration']}\n\n{result['src']}\n\n{result['generation']}"
        elif result['type'] == 'figure':
            if result['path'] is not None:
                replacement_text = f"![图[figure]]({result['path']})\n\n{result['description']}"
            else:
                replacement_text = f"{result['illustration']}\n\n{result['src']}\n\n{result['generation']}"
        elif result['type'] == 'analysis':
            if result['result'] is not None:
                replacement_text = f"{result['description']}"
            else:
                replacement_text = f"{result['illustration']}\n\n{result['src']}\n\n{result['generation']}"
        final_section_text = final_section_text.replace(placeholder, replacement_text)
    # 保存最终的章节内容
    final_subtask_list.append(final_section_text)
    with open(f"subtask_{i}_0924.md", 'w', encoding='utf-8') as f:
        f.write(final_section_text)
# 保存最终的subtask内容
with open('final_subtask_list_0924.pkl', 'wb') as f:
    pickle.dump(final_subtask_list, f)    




--------------------开始处理画图任务 5: 横向竞争分析(商汤科技与同行企业财务表现、业务模式及市场策略比较)  --------------------


You are a helpful assistant that generates HTML tables with my given data.

Generate a complete and self-contained HTML file that displays a clean, modern, and responsive data table. The table should be styled with internal CSS (inside a <style> tag in the <head>) to ensure it looks professional and is ready for screenshotting.

Requirements:
- Use semantic HTML: <table>, <thead>, <tbody>, <th>, <td>.
- Include a clear and descriptive title **ABOVE** the table using <h2> (e.g., "商汤科技收入情况").
- DO NOT include the word "表：" or patterns like "表[table]：" in the title. Just use the natural name (e.g., "商汤科技收入情况", not "表：商汤科技收入情况").
- Include realistic sample data with at least 4 columns and 6 rows.
- Apply modern styling: clean borders, alternating row colors (zebra striping), hover effects, proper padding, and a styled header (e.g., dark background with white text).
- Make the table responsive by wra

In [22]:
extracted_figures[ti]

{'type': '数据分析',
 'src': '商汤2025上半年收入同比增长约64%（年化），增速领先于多数同业，得益于生成式AI业务爆发式增长（2024年同比增长103.1%）；\n  主营利润率仍为负值（-60.28%），主因仍在投入期，成本控制尚未体现规模效应；\n  每股现金流为-0.02元，虽仍为净流出，但较2024年全年-0.11元有所改善，显示运营效率逐步优化。',
 'illustration': '通过数据分析可得，商汤科技2025年上半年收入同比增长64%，显著高于2024年全年增速，其中生成式AI业务已成为核心增长引擎，预计其收入占比已进一步提升至接近70%，推动整体业务结构向高附加值方向演进。',
 'generation': '基于商汤科技2024年全年收入39.99亿元与2025年上半年收入25.86亿元，计算同比增速，并结合生成式AI业务2024年同比增长103.1%的数据，推算该业务在2025H1的贡献占比变化趋势，判断其收入结构转型节奏。'}

In [21]:
gen_table_task

'任务要求：请根据以下内容生成一个HTML格式的表格。\n预期表格描述：表[table]列出了商汤科技与主要可比公司在2024年度的核心财务指标，涵盖营业收入、税后利润、毛利率、净资产收益率及资产负债率，便于横向比较其盈利水平与资本结构健康度。\n原始数据：\n| 公司简称 | 营业收入（亿元） | 税后利润（亿元） | 毛利率 | 净资产收益率（%） | 资产负债率（%） |\n|----------|------------------|------------------|--------|--------------------|----------------|\n| 商汤-W   | 39.99            | -45.66           | 42.9%  | -18.10             | 31.67          |\n| 腾讯控股 | 6,999.38         | 2,082.75         | 47.2%  | 21.51              | 40.83          |\n| 中国软件国际 | 179.70        | 5.42             | 22.1%  | 4.37               | 37.94          |\n| 中国民航信息网络 | 92.53     | 22.57            | 50.6%  | 9.57               | 22.89          |\n| 伟仕佳杰 | 890.86           | 11.16            | 4.4%   | 12.51              | 76.89          |\n| 第四范式 | 55.77            | -3.14            | 42.7%  | -5.09              | 33.17          |\n| 创新奇智 | 12.95            | -6.46            | 34.6%  | -32.68             | 34.73          |\n| 地平线机器人-W | 25.27       | 24.88            | 

# 从task_list生成章节

In [26]:
# 从task_list内容生成章节大标题
generate_section_title_prompt_template = """你是一个专业的金融研究员，正在撰写关于 {stock_info} 的公司研究报告。

该报告将涵盖以下研究任务：
{task}

你已经完成的研究内容如下（按执行顺序列出）：
{task_list_str}

你的任务是：  
基于上述已完成的研究内容，将其归纳为若干个逻辑清晰、层次分明的**一级章节标题**（section title），每个标题应概括一组相关主题，体现研究报告的标准结构和专业性。

要求：  
1. 输出为一个 Python 列表，仅包含序号和章节大标题字符串；  
2. 标题需简洁专业，符合金融行业研报惯例（如“公司概况”、“财务分析”、“估值与投资建议”等）；  
3. 合并相似主题，避免重复或过细划分；  
4. 章节顺序应符合逻辑推理流程：从基础信息 → 业务与行业 → 财务分析 → 竞争格局 → 估值 → 风险 → 投资结论。

示例格式：
["一、公司概况和主营业务","二、财务状况深度分析", "三、同业比较与竞争格局", "四、估值建模与未来预测", "五、风险因素分析", "六、投资建议与总结"]

现在请为 {stock_info} 的研究报告生成章节大标题：
"""

# task_list = ['收集并整理商汤科技（0020.HK）的基本信息，包括公司背景、上市时间、股票代码、交易所等  ',
#  '提取并分析商汤科技的三大会计报表（资产负债表、利润表、现金流量表），整理关键财务数据  ',
#  '梳理商汤科技的主营业务、核心竞争力及行业地位，结合行业报告与公开数据  ',
#  '计算关键财务比率（如ROE、毛利率、现金流匹配度等），并与同行企业进行横向对比分析  ',
#  '分析商汤科技的股权结构与管理层信息，评估公司治理结构与发展战略  ',
#  '构建估值模型（如DCF、PE等），预测未来财务表现，并模拟关键变量（如成本、汇率）的影响  ',
#  '选取同行企业（如旷视科技、云从科技等）进行竞争分析，评估商汤科技的行业竞争优势与劣势  ',
#  '结合财务分析、行业对比及估值结果，提出投资建议（如买入、持有或卖出）  ',
#  '识别并列出商汤科技的主要风险因素（如政策风险、技术风险、市场竞争等），提出风险提醒  ',
#  '汇总所有分析结果，撰写完整的金融研究报告，确保逻辑清晰、数据准确']

generate_section_title_prompt=generate_section_title_prompt_template.format(
    stock_info=input_stock_code,
    task=task1,
    task_list_str='\n'.join([f"{task_list[i]}" for i in range(len(task_list))])
)
section_title_response = qwen_plus.invoke(generate_section_title_prompt)
import ast
try:
    standard_sections = ast.literal_eval(section_title_response.content.strip())
    print("章节大标题列表：", standard_sections)
    #print(type(standard_sections))  # <class 'list'>
    #print(standard_sections[0])     # 公司概况
except (SyntaxError, ValueError) as e:
    print("解析失败：返回内容格式错误", e)

章节大标题列表： ['一、公司概况与主营业务分析', '二、财务状况与核心比率分析', '三、行业地位与竞争格局分析', '四、估值模型与未来业绩预测', '五、公司治理与发展战略评估', '六、风险因素与投资建议']


# 把内容分配到章节

In [27]:
# 把任务解决方案写入到报告中，形成一篇内容翔实的报告

auto_organize_sections_prompt_template = """
你是一个专业的金融研究报告编辑，需要将以下多个子话题的内容归类到标准的金融研究报告结构中。

每个子话题如下：
-----
{task_content_pairs}
-----

请将这些子任务内容归类到以下标准章节中：
{standard_sections}

请输出一个结构化结果，格式如下：
#1# 子任务标题
归类章节：公司概况
内容摘要：该部分内容主要介绍了公司基本信息和股权结构。

#2# 子任务标题
归类章节：财务分析
内容摘要：该部分内容主要分析了三大会计报表和财务比率。

...

要求：
- 不要遗漏任何子任务；
- 不要添加解释或说明；
- 严格按照输出格式；
- 章节名称必须从上面标准章节中选择；
- 内容摘要要简明扼要。
"""

# 生成子任务标题 + 内容的拼接字符串
task_content_pairs_str = ""
for idx in range(len(task_list)):
    task_content_pairs_str += f"#{idx}# {task_list[idx]}\n"

# 构建 Prompt
organize_prompt = auto_organize_sections_prompt_template.format(
    task_content_pairs=task_content_pairs_str.strip(),
    standard_sections='\n'.join(standard_sections)
)

# 调用模型
response = qwen_plus.invoke(organize_prompt,enable_thinking=False)
sleep(1)
organize_result = response.content.strip()
print('归类结果：', organize_result)

归类结果： #0# 收集并整理商汤科技（0020.HK）的基本信息，包括公司背景、上市时间、股票代码、交易所等  
归类章节：一、公司概况与主营业务分析  
内容摘要：该部分内容主要介绍了公司基本信息和上市情况。

#1# 提取并分析商汤科技的三大会计报表（资产负债表、利润表、现金流量表），整理关键财务数据  
归类章节：二、财务状况与核心比率分析  
内容摘要：该部分内容主要分析了三大会计报表中的关键财务数据。

#2# 梳理商汤科技的主营业务、核心竞争力及行业地位，结合行业报告与公开数据  
归类章节：一、公司概况与主营业务分析  
内容摘要：该部分内容主要阐述了公司的主营业务构成与行业定位。

#3# 计算关键财务比率（如ROE、毛利率、现金流匹配度等），并与同行企业进行横向对比分析  
归类章节：二、财务状况与核心比率分析  
内容摘要：该部分内容主要计算并对比分析关键财务比率。

#4# 横向竞争分析(商汤科技与同行企业财务表现、业务模式及市场策略比较)  
归类章节：三、行业地位与竞争格局分析  
内容摘要：该部分内容主要比较了公司在行业内的竞争表现与策略差异。

#5# 构建估值模型（如DCF、PE等），预测未来财务表现，并模拟关键变量（如成本、汇率）的影响  
归类章节：四、估值模型与未来业绩预测  
内容摘要：该部分内容主要建立估值模型并预测未来业绩与敏感性分析。

#6# 股权结构、公司治理与管理层分析  
归类章节：五、公司治理与发展战略评估  
内容摘要：该部分内容主要分析了公司股权结构与治理机制。

#7# 发展战略与未来增长潜力评估  
归类章节：五、公司治理与发展战略评估  
内容摘要：该部分内容主要评估公司战略方向与增长前景。

#8# 结合财务分析、行业对比及估值结果，提出投资建议（如买入、持有或卖出）  
归类章节：六、风险因素与投资建议  
内容摘要：该部分内容综合分析后提出明确的投资建议。

#9# 识别并列出商汤科技的潜在风险因素（如政策风险、技术风险、市场竞争等），提出风险提醒  
归类章节：六、风险因素与投资建议  
内容摘要：该部分内容主要识别并提示公司面临的主要风险因素。


In [28]:
# 归入结果提取
from collections import defaultdict
import re

# 正则提取归类结果
pattern = r'#(\d+)# (.*?)\n归类章节：(.*?)\n内容摘要：(.*?)(?=\n#|\Z)'
matches = re.findall(pattern, organize_result, re.DOTALL)
# 构建结构化归类字典
section_content_map = defaultdict(list)

# matches 是你提供的模型输出结果
for idx, task_title, model_section, summary in matches:
    # 清洗模型输出的章节名（去除前后空格/换行）
    cleaned_section = model_section.strip()

    # 初始化匹配章节为 None
    matched_section = None

    # 模糊匹配标准章节
    for std_sec in standard_sections:
        if std_sec in cleaned_section or cleaned_section in std_sec:
            matched_section = std_sec
            break

    # 如果完全没匹配上，归类为 "其他分析"
    if not matched_section:
        matched_section = "其他分析"

    # 清洗任务标题（去除前后空格）
    cleaned_task_title = task_title.strip()

    # 找到对应的完整内容
    full_content = None
    for task, content in zip(task_list, task_solution_list):
        if cleaned_task_title in task or task in cleaned_task_title:
            full_content = content.strip()
            break

    if full_content:
        section_content_map[matched_section].append(idx)

section_content_map

defaultdict(list,
            {'一、公司概况与主营业务分析': ['0', '2'],
             '二、财务状况与核心比率分析': ['1', '3'],
             '三、行业地位与竞争格局分析': ['4'],
             '四、估值模型与未来业绩预测': ['5'],
             '五、公司治理与发展战略评估': ['6', '7'],
             '六、风险因素与投资建议': ['8', '9']})

# 章节草稿

In [30]:
full_report = f"# {input_stock_code} 金融研究报告\n\n"

seen_content = set()

# 按照标准章节顺序拼接
for section_title in standard_sections:
    idx = section_content_map.get(section_title)
    # 获取对应章节的内容
    contents = []
    if idx:
        for num in idx:
            # 注意这里的i是字符串，需要转换为整数
            task_index = int(num) - 1  # 索引从0开始
            if task_index < len(final_subtask_list):
                content = final_subtask_list[task_index].strip()
                if content:  # 如果内容不为空
                    contents.append(content)
    if contents:
        full_report += f"## {section_title}\n\n"
        for content in contents:
            clean_content = re.sub(r'\s+', ' ', content).strip()
            if clean_content not in seen_content:
                full_report += content + "\n\n"
                seen_content.add(clean_content)

# 保存为 Markdown 文件
with open("company_report_final.md", "w", encoding="utf-8") as f:
    f.write(full_report)

print("✅ company_report_final.md")

✅ company_report_final.md


# 章节润色

In [ ]:
# 对每个章节的内容进行润色，一方面是围绕主题来写，纠正幻觉
# polish_section_prompt_template = """
# 你是一个专业的金融研究报告编辑，负责对本章节内容进行润色与整合，目标是输出一个**逻辑清晰、语言专业、无幻觉**的完整章节内容，以markdown文本格式输出。

# 【报告主题】
# -----
# {query}
# -----

# 【完整报告目录结构】
# -----
# {report_structure}
# -----

# 【当前章节标题】
# -----
# {section_title}
# -----

# 【当前章节的原始内容】
# -----
# {section_content}
# -----

# 【润色要求】
# 1. **围绕当前章节标题**组织内容，确保内容紧扣主题；
# 2. **整理好排版，输出内容应包含当前章节标题**
# 3. **去掉原始内容中多余的空格、换行符和格式混乱的部分**；
# 4. **理清本章节在整体报告中的逻辑定位**，整合原始内容，删除冗余信息，合并重复内容，不遗漏重要信息和表述；
# 5. **确保语言正式、专业**，符合金融研究报告的写作规范；
# 6. **纠正可能存在的数据错误、逻辑矛盾或幻觉内容**；
# 7. **仅基于原始内容进行润色，不添加任何原始内容中没有的信息或数据**；
# 8. 把原始内容中的"图[figure]"和"表[table"]替换为准确的序号，每个章节的第一个图表从计数1开始，使用章节.计数的形式，例如第一章的第一幅图是图1.1，不要删掉图表的超链接否则显示不出来
# 9. **输出格式为完整的markdown格式的研究报告章节**，不要包含解释性语句（如“我将如何整合”）；

# 特别的，对于当前章节的标题和子标题、列表等，你需要尤其注意格式：
# 1. 当前章节标题{section_title}为一级标题（#）格式，例如"# 一、"和"# 二、"
# 2. 其他的子标题不允许使用"# 一、"和"# 二、"这样的中文序号标题格式，可以使用"(一)"或者"(二)"这样的格式，或者使用"1."、"2."这样的数字序号格式等等
# 2. 对于原始内容进行排版，如果原始内容里面出现了某个子标题内容特别长，需要对其进行浓缩符合子标题风格

# 请直接输出润色后的章节内容：
# """

polish_section_prompt_template = """
你是一个专业的金融研究报告编辑，负责对本章节内容进行润色与整合，目标是输出一个**逻辑清晰、语言专业、无幻觉、无代码**的最终版研究报告章节，以纯 markdown 文本格式输出。

【报告主题】
-----
{query}
-----

【完整报告目录结构】
-----
{report_structure}
-----

【当前章节标题】
-----
{section_title}
-----

【当前章节的原始内容】
-----
{section_content}
-----

【润色要求】
1. **围绕当前章节标题**组织内容，确保内容紧扣主题；
2. **整理排版，输出内容应包含当前章节标题**；
3. **清除原始内容中的多余空格、换行符、格式混乱、重复段落或工具调用指令**（如“生成图表”、“调用 chart_gen”等中间步骤）；
4. **理清本章节在整体报告中的逻辑定位**，整合内容，删除冗余，合并重复，不遗漏重要信息；
5. **确保语言正式、专业**，符合金融研究报告的写作规范；
6. **纠正可能存在的数据错误、逻辑矛盾或幻觉内容**；
7. **仅基于原始内容进行润色，不添加任何原始内容中没有的信息、数据或代码**；
8. **特别注意：**
   - 原始内容中可能包含已成功生成的图表或表格，其形式为 `![图 X.X](...)` 或 `![表 X.X](...)` 或 `![chart](data:image/png;base64,...)` 等 **Markdown 图片链接**；
   - 这些 **图表链接必须原样保留，不得删除、修改或替换为代码**；
   - 如果原始内容中包含“图[figure]”或“表[table]”等占位符，请将'[figure]'和'[table]'替换为正确的编号格式（如 `图1.1`、`表1.1`）；
   - **严禁生成任何 HTML、JavaScript、Chart.js 代码或代码块**；
   - 如果生成图表的指令内容出现在原始内容中，说明这个图表生成失败了，请删除相关指令内容，根据数据改写为文字描述即可
9. **图表与表格编号规则：**
   - 每个章节独立编号，从 1 开始；
   - 格式为：**图[章].[序数]**、**表[章].[序数]**（例如：图1.1、图1.2、表1.1）；
   - 替换所有“图[figure]”为“图1.1”等形式
10. **输出格式要求：**
    - 输出为 **纯 markdown 文本**；
    - 包含当前章节标题（一级标题 `#`）；
    - 子标题使用 `(一)`、`(二)` 或 `1.`、`2.` 等格式，**禁止使用 `# 一、`、`# 二、` 等中文序号一级标题格式**；
    - 长子标题需浓缩为简洁标题；
    - **不要包含任何解释性语句**（如“我将如何整合”、“以下是润色后的内容”等）；
    - **不要输出代码块**（如 ```html、```js 等）；
    - **不要生成新图表或表格，只保留已有链接**；

请直接输出润色后的最终章节内容（纯 markdown 格式）：
"""


polished_result_list = []
# 按照标准章节顺序拼接
for i in range(len(standard_sections)):
    section_title = standard_sections[i]  # 当前章节标题
    print(f"正在润色章节：{section_title}")
    idx = section_content_map.get(section_title)
    # 获取对应章节的内容
    contents = []
    if idx:
        for num in idx:
            # 注意这里的i是字符串，需要转换为整数
            task_index = int(num) - 1  # 索引从0开始
            if task_index < len(final_subtask_list):
                content = final_subtask_list[task_index].strip()
                if content:  # 如果内容不为空
                    contents.append(content)
    if contents:
        polish_section_prompt = polish_section_prompt_template.format(
            query=task1,  # 输入的研究任务
            report_structure='\n'.join(standard_sections),  # 报告结构
            section_title=standard_sections[i],  # 当前章节标题
            section_content='\n\n'.join(contents)  # 当前章节内容
        )
        # 调用模型进行润色
        polish_response = qwen_plus.invoke(polish_section_prompt)
        sleep(2)
        polished_content = polish_response.content.strip()
        polished_result_list.append(polished_content)  # 存储润色后的内容

# pickle保存润色后的内容
import pickle
with open('comany_polished_result_list.pkl', 'wb') as f:
    pickle.dump(polished_result_list, f)

正在润色章节：一、公司概况与主营业务分析
正在润色章节：二、财务状况与核心比率分析
正在润色章节：三、行业地位与竞争格局分析
正在润色章节：四、估值模型与未来业绩预测
正在润色章节：五、公司治理与发展战略评估
正在润色章节：六、风险因素与投资建议


In [ ]:
#10. **保留原始内容中的"图[figure]"和"表[table]占位符"**，不要删除或修改它们为标号，除非判断对应的图表内容不合适当前章节主题，可以删除该占位符和对应的图、表及相关描述内容；

# 最终markdown结果输出

In [36]:
# 生成最终的报告
final_report = f"# {input_stock_code} 金融研究报告\n\n"
for content in polished_result_list:
    final_report += content + "\n\n"
# 保存为 Markdown 文件
final_report = final_report.replace('```markdown', '').replace('```', '').replace('/Users/xwdong/Documents/阿里天池算法比赛/AFAC2025比赛_多模态金融报告agent生成/赛后完善', '.')
with open("company_report_polished_final.md", "w", encoding="utf-8") as f:
    f.write(final_report)

In [ ]:
import markdown
from playwright.async_api import async_playwright
import os

async def markdown_to_pdf_async(md_content, output_pdf_path):
    # ✅ 使用 extensions=['tables'] 支持表格
    html_content = f"""
    <html>
    <head>
        <meta charset="UTF-8">
        <style>
            body {{ 
                font-family: "Microsoft YaHei", "SimSun", sans-serif; 
                margin: 40px; 
            }}
            table {{
                border-collapse: collapse;
                width: 100%;
                margin: 10px 0;
            }}
            th, td {{
                border: 1px solid #ccc;
                padding: 8px;
                text-align: left;
            }}
            th {{
                background-color: #f2f2f2;
                font-weight: bold;
            }}
            pre {{ 
                background-color: #f4f4f4; 
                padding: 10px; 
                border-radius: 5px; 
                overflow-x: auto;
            }}
            code {{ 
                font-family: Consolas, monospace; 
            }}
        </style>
    </head>
    <body>
        {markdown.markdown(md_content, extensions=['tables', 'fenced_code'])}
    </body>
    </html>
    """

    temp_html = "temp_markdown.html"
    with open(temp_html, "w", encoding="utf-8") as f:
        f.write(html_content)

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(f"file://{os.path.abspath(temp_html)}")
        await page.pdf(
            path=output_pdf_path,
            format='A4',
            margin={"top": "1in", "bottom": "1in", "left": "1in", "right": "1in"}
        )
        await browser.close()

    os.remove(temp_html)
    print(f"✅ PDF 已生成: {output_pdf_path}")

with open("company_report_polished_final.md", "r", encoding="utf-8") as f:
    md_text = f.read()

await markdown_to_pdf_async(md_text, "output.pdf")

In [ ]:
# # 把图[figure]和表[table]占位符替换为对应的标号
# figure_cnt = 1
# table_cnt = 1
# for i in range(len(polished_result_list)):
#     content = polished_result_list[i]
#     # 替换图[figure]占位符
#     def replace_figure(match):
#         global figure_cnt
#         replacement = f"图{figure_cnt}"
#         figure_cnt += 1
#         return replacement

#     content = re.sub(r'图\[figure\]', replace_figure, content)

#     # 替换表[table]占位符
#     def replace_table(match):
#         global table_cnt
#         replacement = f"表{table_cnt}"
#         table_cnt += 1
#         return replacement

#     content = re.sub(r'表\[table\]', replace_table, content)

#     polished_result_list[i] = content
# # 生成最终的报告
# final_report = f"# {input_stock_code} 金融研究报告\n\n"
# for content in polished_result_list:
#     final_report += content + "\n\n"
# # 保存为 Markdown 文件
# with open("company_report_polished_final.md", "w", encoding="utf-8") as f:
#     f.write(final_report)

In [ ]:
# final_report = f"# {input_stock_code} 金融研究报告\n\n"
# for content in polished_result_list:
#     final_report += content + "\n\n"
# # 保存为 Markdown 文件
# with open("company_report_polished_final.md", "w", encoding="utf-8") as f:
#     f.write(final_report)

In [ ]:
# from python_data_analyse import *
# if True:
#     with Sandbox.create(api_key=E2B_API_KEY) as code_interpreter:
#         task_prompt = data_anylyse_prompt_template.format(task=gen_table_task)

#         code_results,python_code = chat_with_llm(
#             code_interpreter,
#             task_prompt
#         )
#         if code_results[0]==0:  # 执行成功
#             first_result = code_results[1][0]

# from python_data_analyse import screenshot_with_playwright
# task_cnt = 0
# current_attempt=0
# await screenshot_with_playwright_async(f"{current_path}/{task_cnt}_{current_attempt+1}.html", 'div', f"{current_path}/{task_cnt}_{current_attempt+1}.png")

# # matplotlib画图
# from python_data_analyse import *
# if True:
#     with Sandbox.create(api_key=E2B_API_KEY) as code_interpreter:
#         task_prompt = data_anylyse_prompt_template.format(task=gen_table_task)

#         code_results,python_code = chat_with_llm(
#             code_interpreter,
#             task_prompt
#         )
#         if code_results[0]==0:  # 执行成功
#             first_result = code_results[1][0]


# # 生成表格
# task_cnt = 0
# if True:
#     max_attempts = 3
#     for current_attempt in range(max_attempts):
#         if current_attempt==0:
#             html_table_gen_prompt = html_table_gen_template.format(task=gen_table_task)
#         else:
#             html_table_gen_prompt = html_table_review_gen_template.format(task=gen_table_task,
#                                                                         pre_code=html_table_response.content,
#                                                                         vlm_feedback=vlm_feedback)
#         print(html_table_gen_prompt)
#         html_table_response = qwen_plus.invoke(html_table_gen_prompt)
#         with open(f"{task_cnt}_{current_attempt+1}.html", "w") as f:
#             f.write(html_table_response.content)

#         # --- 使用 ---
#         await screenshot_with_playwright_async(f"{current_path}/{task_cnt}_{current_attempt+1}.html", 'div', f"{task_cnt}_{current_attempt+1}.png")

#         # 质检
#         vlm_feedback = vlm_check_function(f"{task_cnt}_{current_attempt+1}.png",task=gen_table_task)

#         if "PASS" in vlm_feedback:
#             print("VLM quality check passed.")
#             break
#         else:
#             print("VLM quality check failed.")
#         print(vlm_feedback)


# # html画图
# html_pic_outout = 0
# if True:
#     max_attempts = 3
#     for current_attempt in range(max_attempts):
#         if current_attempt==0:
#             html_pic_gen_prompt = html_pic_gen_template.format(task=gen_table_task)
#         else:
#             html_pic_gen_prompt = html_pic_review_gen_template.format(task=gen_table_task,
#                                                                        pre_code=html_table_response.content,
#                                                                        vlm_feedback=vlm_feedback)
#         print(html_pic_gen_prompt)
#         html_table_response = qwen_plus.invoke(html_pic_gen_prompt)
#         with open(f"{html_pic_outout}_{current_attempt+1}.html", "w") as f:
#             f.write(html_table_response.content)

#         # step2 截图
#         await screenshot_with_playwright_async(f"{current_path}/{html_pic_outout}_{current_attempt+1}.html", 'div', f"{current_path}/{html_pic_outout}_{current_attempt+1}.png")
#         # step3  质检
#         vlm_feedback = vlm_check_function(f"{html_pic_outout}_{current_attempt+1}.png", task1)

#         if "PASS" in vlm_feedback:
#             print("VLM quality check passed.")
#             break
#         else:
#             print("VLM quality check failed.")

#         print(vlm_feedback)